In [ ]:
import pydot
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from networkx.drawing.nx_pydot import graphviz_layout

In [ ]:
class Graph:
    def __init__(self, vertices, edges, start, goal, weighted = False):
        '''
        edge is a tuple of vertices (node 1, node 2, cost)
        think this is already kind of a digraph?
        '''
        self.vertices, self.edges = {}, {}
        self.weighted = weighted
        self.visualization = edges
        self.start, self.goal = start, goal
        self.adjList = defaultdict(list)
        self.initialize_params(vertices, edges)
    
    def visualize_graph(self):
        nx_graph = nx.Graph()
        if (self.weighted == True):
            nx_graph.add_weighted_edges_from(self.visualization)
            costs = nx.get_edge_attributes(nx_graph, 'weight')
            pos = graphviz_layout(nx_graph, prog = "dot", root = self.start)
            nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels = costs, font_color = 'red')
            nx.draw_networkx(nx_graph, pos, with_labels = "True", alpha = 0.9, node_color = 'cyan')
        else:
            nx_graph.add_edges_from(self.visualization)
            pos = graphviz_layout(nx_graph, prog = "dot", root = self.vertices[0])
            nx.draw_networkx(nx_graph, pos, with_labels = "True")
        plt.show()
        
    def create_adj_list(self, edges):
        for edge in edges:
            source = edge[0]
            destination = edge[1]
            self.adjList[source].append(destination)
            self.adjList[destination].append(source)
            
    def print_adj_list(self):
        for item in self.adjList.items():
            print(item)
    
    def create_node_dict(self, vertices):
        for vertex in vertices:
            node, heuristic = vertex
            self.vertices[node] = heuristic
    
    def create_edge_dict(self, edges):
        for edge in edges:
            edge_name = (edge[0], edge[1])
            try:
                edge_cost = edge[2]
            except:
                edge_cost = 0
            self.edges[edge_name] = edge_cost
    
    def initialize_params(self, vertices, edges):
        self.create_adj_list(edges)
        self.create_node_dict(vertices)
        self.create_edge_dict(edges)

In [ ]:
from collections import deque
from copy import deepcopy

def get_path(current_position, previous_node):
    path = []
    while current_position in previous_node.keys() and current_position != None:
        path.append(current_position)
        current_position = previous_node[current_position]
    return path

def BFS(graph):
    '''
    Returns a frontier history, explore history, and path history. 
    Use history to visualize the frontier, path, and explored nodes at each step. 
    '''
    frontier_steps = []
    explored_steps = []
    start = graph.start
    goal = graph.goal
    explored = deque([start])
    frontier = deque([start])
    previous_node = {start: None}
    paths_history = []
    frontier_steps.append(deepcopy(frontier))
    explored_steps.append(deepcopy(explored))
    paths_history.append([])
    while len(frontier) != 0:
        current_node = frontier[0]
        frontier.popleft()
        if current_node == goal:
            frontier_steps.append(deepcopy(frontier))
            explored_steps.append(deepcopy(explored))
            paths_history.append(get_path(current_node, previous_node))
            return frontier_steps, explored_steps, paths_history       
        for node in graph.adjList[current_node]:
            if node not in explored:
                frontier.append(node)
                explored.append(node)
                previous_node[node] = current_node
        frontier_steps.append(deepcopy(frontier))
        explored_steps.append(deepcopy(explored))
        paths_history.append(get_path(current_node, previous_node))

In [ ]:
def DFS(graph):
    '''
    Returns a frontier history, explore history, and path history. 
    Use history to visualize the frontier, path, and explored nodes at each step. 
    '''
    frontier_steps = []
    explored_steps = []
    start = graph.start
    goal = graph.goal
    explored = deque([start])
    frontier = deque([start])
    previous_node = {start: None}
    paths_history = []
    frontier_steps.append(deepcopy(frontier))
    explored_steps.append(deepcopy(explored))
    paths_history.append([])
    while len(frontier) != 0:
        current_node = frontier[-1]
        frontier.pop()
        if current_node == goal:
            frontier_steps.append(deepcopy(frontier))
            explored_steps.append(deepcopy(explored))
            paths_history.append(get_path(current_node, previous_node))
            return frontier_steps, explored_steps, paths_history       
        for node in graph.adjList[current_node]:
            if node not in explored:
                frontier.append(node)
                explored.append(node)
                previous_node[node] = current_node
        frontier_steps.append(deepcopy(frontier))
        explored_steps.append(deepcopy(explored))
        paths_history.append(get_path(current_node, previous_node))